# Generic

In [5]:
import math
import base64
import binascii

def to_bytes(d, format):
    if isinstance(d, (bytes, bytearray)):
        return d
    elif format == 'hex':
        return bytes(bytearray.fromhex(d))
    elif format == 'base64':
        return base64.b64decode(d)
    elif format == 'str' or format == 'bytes':
        return d.encode()
    elif format == 'int':
        return to_bytes(hex(d)[2:], 'hex')

def bytes_to(b, format):
    if not isinstance(b, (bytes, bytearray)):
        return b
    elif format == 'hex':
        return binascii.hexlify(b).decode()
    elif format == 'base64':
        return base64.b64encode(b).decode()
    elif format == 'str':
        return b.decode()
    elif format == 'bytes':
        return b
    elif format == 'int':
        return int(bytes_to(b, 'hex')[2:], 16)

In [6]:
import math
import itertools

def xor(d1, d2, format):
    b1,b2 = to_bytes(d1, format), to_bytes(d2, format)
    return bytes_to(bytes(char1 ^ char2 for char1,char2 in zip(b1,b2)), format)

def pad(text, blocksize=16):
    padsize = blocksize - (len(text) % blocksize)
    if padsize == 0:
        padsize = blocksize
    return text + bytes([padsize]*padsize)
assert pad(b"YELLOW SUBMARINE", blocksize=20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'

def unpad(text):
    """Validate padding and return unpadded value"""
    try:
        assert text[-text[-1]:] == bytes([text[-1]]*text[-1])
    except:
        print(text)
        raise
    return text[:-text[-1]]
assert unpad(pad(b"this is random text", blocksize=7)) == b"this is random text"

# Crypto

In [7]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend


def decrypt_aes_ecb(enc, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()
    clear = decryptor.update(enc) + decryptor.finalize()
    if use_padding:
        return unpad(clear)
    else:
        return clear

def encrypt_aes_ecb(clear, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    if use_padding:
        clear = pad(clear)
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    ct = encryptor.update(clear) + encryptor.finalize()
    return ct
assert decrypt_aes_ecb(encrypt_aes_ecb(b'test', b'YELLOW SUBMARINE'), b'YELLOW SUBMARINE') == b'test'

def encrypt_aes_cbc(clear, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = pad(clear) # Ensure the size is divisible by 16
    prev_block = IV
    cipher = b''
    for blockstart in range(0, len(clear), 16):
        block = clear[blockstart:blockstart+16]
        cipher_block = encrypt_aes_ecb(xor(prev_block, block, 'bytes'), key, use_padding=False)
        prev_block = cipher_block
        cipher += cipher_block
    return cipher

def decrypt_aes_cbc(cipher, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = b''
    prev_block = IV
    for blockstart in range(0, len(cipher), 16):
        block = cipher[blockstart:blockstart+16]
        tmp = decrypt_aes_ecb(block, key, use_padding=False)
        clear_block = xor(prev_block, tmp, 'bytes')
        prev_block = cipher[blockstart:blockstart+16]
        clear += clear_block
    return unpad(clear)
assert decrypt_aes_cbc(encrypt_aes_cbc(b'test'*200, b'YELLOW SUBMARINE'), b'YELLOW SUBMARINE') == b'test'*200

def MAC(message, key, IV):
    """Simple MAC calculation.
    """
    return encrypt_aes_cbc(message, key, IV)[-16:]

# Cryptopals Notebook

## Set 7

### Exercice 49

CBC-MAC Message Forgery

In [8]:
# Setup : a client and a server share a key K.
# The server has methods to validate an incoming message from the client

import os, random

class Server:
    def __init__(self, key):
        self._key = key
        
    def validate_simple(self, message, iv, mac):
        """For the simple çase with a user-provided IV"""
        return MAC(message, self._key, iv) == mac
    
    def validate(self, message, mac):
        """Using a fixed IV"""
        return MAC(message, self._key, b'\x00'*16) == mac
    
class Client:
    def __init__(self, key):
        self._key = key
    
    def emit_simple(self, amount):
        """Emit a message without fixed IV, that will be intercepted by the attacker"""
        message = 'from=bob&to=alice&amount={}'.format(amount).encode()
        iv = os.urandom(16)
        return message, iv, MAC(message, self._key, iv)
    
    def emit(self):
        """Emit a message using a fixed IV, with a list of transactions."""
        message = b'from=client&tx_list=customer0:17'
        for i in range(9):
            message += (';customer' + str(i+1) + ':' + str(random.randint(1, 20))).encode()
        iv = b'\x00'*16
        return message, MAC(message, self._key, iv)
    
def init():
    key = os.urandom(16)
    return Server(key), Client(key)

In [9]:
alice, bob = init()
assert alice.validate_simple(*bob.emit_simple(10))
assert alice.validate(*bob.emit())

In [28]:
# First : the simple case, with a controlled IV
message, iv, mac = bob.emit_simple(1000000)

# Change message :
new_message = b'  from=bob&to=eve&amount=1000000'
iv = xor(xor(new_message[:16], iv, 'bytes'), message[:16], 'bytes')
assert alice.validate_simple(new_message, iv, mac)

We will choose as an IV the following value :
$$\begin{align*}
IV
&= message_{block1} \bigoplus IV_{old} \bigoplus new\_message_{block1} 
\end{align*}$$

Now to the trickier part. The idea is to realise that the MAC is actually just the last block of the CBC cipher. This means that if we expand the message by adding to the end of it, the MAC will appear somewhere in the middle.

In [27]:
iv

b'\rS\xc1\x99\xa4\xf9\xd5\xb6\x03\xd2\xf1\xbe~\x7f\x9e\x05'